### Setup and imports

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Bayesian MCMC
import pymc as pm
import arviz as av

# Genetic algorithm
from deap import base, creator, tools, algorithms

# CMA-ES
import cma

script_dir = Path(__file__).parent

### Load & Flatten the theme

In [ ]:
# Load our palette.json
palette_path = script_dir / ".." / "palettes" / "catpuccin" / "palette.json"

def palette_to_df(path_to_json: Path) -> pd.DataFrame:
    # Load the full palette
    palette = json.loads(path_to_json.read_text(encoding="utf-8"))
    
    # Build a dict mapping each variant name to its color sub-dict
    variant_colors = {
        name: info["colors"]
        for name, info in palette.items()
        if name != 'version'
    }
    
    # Turn that into a big MultiIndex DataFrame: rows=(variant, token)
    palette_df = pd.concat(
        {
            variant: pd.DataFrame.from_dict(colors, orient="index")
            for variant, colors in variant_colors.items()
        },
        axis=0
    )
    palette_df.index.set_names(['variant', 'token'], inplace=True)
    palette_df = palette_df.reset_index()
    
    # Flatten the nested HSL column
    hsl = pd.json_normalize(palette_df['hsl'].to_dict()).rename(
        columns={
            "h": "h_raw",
            "s": "s",
            "l": "l"
        }
    )
    
    # Normalize hue and assemble final frame
    hsl['h'] = hsl['h_raw'] / 360
    out = pd.concat([
        palette_df[['variant', 'token']].reset_index(drop=True),
        hsl[['h', 's', 'l']],
        palette_df['accent'].reset_index(drop=True)
    ], axis=1)
    
    return out

df = palette_to_df(palette_path)
display(df)